In [1]:
import pandas as pd
import numpy as np

In [2]:
soubor = pd.read_csv('clean_data_years_filter.csv',  encoding='utf-8')
soubor.head(2)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
0,2011-01-01,LEKAR,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,10.0,82.83,828.3,33.3333,333.0,F
1,2011-01-01,LEKARNA,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,71070.0,82.83,5886728.1,33.3333,2368998.0,F


###### "clean_data_years_filter.csv" obsahuje data, kde:

###### "Typ odběratele":
'LEKARNA', 'VETERINARNI_LEKAR', 'PRODEJCE_VLP','HYGIENICKA_STANICE', 'ZDRAVOTNICKE_ZARIZENI', 'LEKAR', 'OSOBA_POSKYTUJICI_ZDRAVOTNI_PECI'

###### Abychom nic nepokazily, vytvorime novou tabulku "soubor1" a přejmenujeme poslední sloupec

In [3]:
soubor1 = soubor.loc[:,['Období', 'ATC7', 'Počet denních definovaných dávek/balení celkem']]

In [4]:
soubor1 = soubor1.rename(index=str, columns={'Počet denních definovaných dávek/balení celkem': 'DDD_total'})
soubor1.head()

,Období,ATC7,DDD_total
0,2011-01-01,M01AE01,333.0
1,2011-01-01,M01AE01,2368998.0
2,2011-01-01,M01AE01,-45533.0
3,2011-01-01,M01AE01,600.0
4,2011-01-01,M01AE01,20.0


### Jako další krok bylo třeba seskupit všechny léky obsahující stejnou molekulu ATC. 
Stejné ATC totiž může být základem pro různé léky. Použily jsme metodu groupby, pomocí které jsme vytvořily součet napříč jednotlivými ATC za každý měsíc. Protože k tomuto výpočtu z původních patnácti sloupců v tabulce nám vystačí tři, jsme do syntaxe zapsaly jen “Období”, “ATC” a “DDD_total”. 

### Zkusime spojit jednotlive ATC do vyssi urovni molekuly. 

Proto napred socteme vsechny ATC, nazev kterych se zacina s R05:

Jsou:  R05','R05C','R05CA','R05CA01','R05CA02', 'R05CA03', 'R05CA04', 'R05CA05', 'R05CA06', 'R05CA07', 'R05CA08', 'R05CA09', 'R05CA10', 'R05CA11', 'R05CA12', 'R05CA13', 'R05CB', 'R05CB01', 'R05CB02', 'R05CB03', 'R05CB04', 'R05CB05', 'R05CB06', 'R05CB07', 'R05CB08', 'R05CB09', 'R05CB10', 'R05CB11', 'R05CB13', 'R05CB14', 'R05CB15', 'R05CB16', 'R05D', 'R05DA', 'R05DA01', 'R05DA03', 'R05DA04', 'R05DA05', 'R05DA06', 'R05DA07', 'R05DA08', 'R05DA09', 'R05DA10', 'R05DA11', 'R05DA12', 'R05DA20', 'R05DB', 'R05DB01', 'R05DB02', 'R05DB03', 'R05DB04', 'R05DB05', 'R05DB07', 'R05DB09', 'R05DB10', 'R05DB11', 'R05DB12', 'R05DB13', 'R05DB14', 'R05DB15', 'R05DB16', 'R05DB17', 'R05DB18', 'R05DB19', 'R05DB20','R05DB21', 'R05DB22', 'R05DB23', 'R05DB24', 'R05DB25', 'R05DB26', 'R05DB27', 'R05DB28', 'R05F', 'R05FB','R05FB01', 'R05FB02', 'R05X'







In [5]:
soubor_R05 = soubor1[soubor1['ATC7'].str.startswith('R05')]

In [6]:
soubor_R05.tail(3)

,Období,ATC7,DDD_total
87763,2018-03-01,R05X,NaN
87764,2018-03-01,R05X,NaN
87765,2018-03-01,R05X,NaN


In [7]:
leky_R05 = soubor_R05.groupby(['Období']).agg({'Období':'first', 'DDD_total':'sum'})
leky_R05.tail()

,Období,DDD_total
Období,,
2017-11-01,2017-11-01,5.855393e+06
2017-12-01,2017-12-01,6.414107e+06
2018-01-01,2018-01-01,7.562418e+06
2018-02-01,2018-02-01,1.114885e+07
2018-03-01,2018-03-01,8.664688e+06


In [8]:
leky_R05 = leky_R05.reset_index(drop=True)

In [9]:
R05_DDD = leky_R05

In [10]:
# Exportujeme
#leky_R05.to_csv('leky/leky_R05.csv', encoding = 'utf-8')

In [11]:
leky_R05['Období'] = pd.to_datetime(leky_R05['Období'])

In [12]:
leky_R05['year'] = leky_R05['Období'].dt.year.astype(np.int)

In [13]:
leky_R05['month'] = leky_R05['Období'].dt.month.astype(np.int)

In [14]:
leky_R05['quater'] = leky_R05['Období'].dt.quarter.astype(np.int)
#DatetimeIndex.quarter

#### Pro statistický model jsme potřebovaly vytvořit “trend”. Náš nejstarší rok je označen nulou a každý další je o jednotku vyšší.

In [15]:
leky_R05['trend'] = leky_R05['year']-2011

In [16]:
leky_R05.tail(3)

,Období,DDD_total,year,month,quater,trend
84,2018-01-01,7.562418e+06,2018,1,1,7
85,2018-02-01,1.114885e+07,2018,2,1,7
86,2018-03-01,8.664688e+06,2018,3,1,7


### Najdeme stredni hodnotu

In [17]:
Jan = leky_R05['DDD_total'][leky_R05['month'] == 1].mean()
Jan

8336916.086312501

In [18]:
months = [0]*13

In [19]:
for i in range (1,13):
    months[i] = leky_R05['DDD_total'][leky_R05['month'] == i].mean()
months = months[1:13]
months

[8336916.086312501,
 8602066.26315,
 6028651.7403125,
 3661711.7329,
 2946264.1526,
 2474747.2884857142,
 2043413.2984571431,
 3123329.507871429,
 7638414.087585716,
 6116561.951985715,
 5422315.419714285,
 5533989.387157144]

In [20]:
average = sum(months) / float(len(months)) 

In [21]:
neco = [ abs(x-average) for x in months ]
neco

[3176217.676601488,
 3441367.8534389865,
 867953.3306014873,
 1498986.6768110124,
 2214434.2571110125,
 2685951.1212252984,
 3117285.1112538697,
 2037368.9018395836,
 2477715.677874703,
 955863.5422747023,
 261617.01000327244,
 373290.9774461314]

In [22]:
neco.index(min(neco))+1

11

### Statistický model
Pro použití statistického modelu jsme potřebovaly vytvořit další sloupce pro rok, měsíc a kvartál pomocí metody knihovny pandas “.to_datetime”.

In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels as smt
import matplotlib.pyplot as plt

C:\Users\Katerina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Program vnímá čísla měsíců (1–12) a kvartálů (1–4) jako hodnoty, což se nedá využít pro náš stats model. Aby program nevnímal číslo měsíců a kvartálů jako hodnotu, je třeba použít binární systém jedniček a nul.

### Proto použijeme metodu “.get_dummies” na sloupce “month” a “quater”.

In [24]:
Q01 = pd.get_dummies(leky_R05['quater'], prefix='quater')
Q01.head()

,quater_1,quater_2,quater_3,quater_4
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,0,1,0,0
4,0,1,0,0


In [25]:
M01 = pd.get_dummies(leky_R05['month'], prefix='month')
M01.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0


### Ted to spojime do jednoho dataframu "model_xx"
https://pandas.pydata.org/pandas-docs/stable/merging.html

In [26]:
model_R05 = pd.concat([leky_R05, M01, Q01], axis=1, join_axes=[leky_R05.index])
model_R05.head(3)

,Období,DDD_total,year,month,quater,trend,month_1,month_2,month_3,month_4,...,month_7,month_8,month_9,month_10,month_11,month_12,quater_1,quater_2,quater_3,quater_4
0,2011-01-01,8326992.0,2011,1,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2011-02-01,8690387.0,2011,2,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2011-03-01,6590397.0,2011,3,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [27]:
results_R05 = smf.ols('DDD_total ~ trend + quater_1 + quater_2 + quater_3', data=model_R05).fit()
print(results_R05.summary())

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.527
Model:                            OLS   Adj. R-squared:                  0.504
Method:                 Least Squares   F-statistic:                     22.81
Date:                Thu, 28 Jun 2018   Prob (F-statistic):           1.09e-12
Time:                        15:08:06   Log-Likelihood:                -1370.0
No. Observations:                  87   AIC:                             2750.
Df Residuals:                      82   BIC:                             2762.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    5.44e+06   4.59e+05     11.841      0.0

In [28]:
resultsR05 = smf.ols('DDD_total ~ trend + month_1 + month_2 + month_3 + month_4 + month_5 + month_6 + month_7 + month_8 + month_9 + month_10 + month_12', data=model_R05).fit()
print(resultsR05.summary())
#np.log

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     33.31
Date:                Thu, 28 Jun 2018   Prob (F-statistic):           5.50e-25
Time:                        15:08:06   Log-Likelihood:                -1321.7
No. Observations:                  87   AIC:                             2669.
Df Residuals:                      74   BIC:                             2702.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.171e+06   4.24e+05     12.185      0.0

In [29]:
arima_R05 = smt.tsa.arima_model.ARIMA(endog = model_R05.DDD_total, order = (1,0,1), dates = model_R05.Období).fit()
print(arima_R05.summary())

                              ARMA Model Results                              
Dep. Variable:              DDD_total   No. Observations:                   87
Model:                     ARMA(1, 1)   Log Likelihood               -1379.498
Method:                       css-mle   S.D. of innovations        1856067.354
Date:                Thu, 28 Jun 2018   AIC                           2766.997
Time:                        15:08:06   BIC                           2776.861
Sample:                    01-01-2011   HQIC                          2770.969
                         - 03-01-2018                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5.247e+06   4.53e+05     11.587      0.000    4.36e+06    6.13e+06
ar.L1.DDD_total     0.4113      0.130      3.155      0.002       0.156       0.667
ma.L1.DDD_total     0.3539      

C:\Users\Katerina\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:646: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if issubdtype(paramsdtype, float):
C:\Users\Katerina\Anaconda3\lib\site-packages\statsmodels\tsa\kalmanf\kalmanfilter.py:650: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  elif issubdtype(paramsdtype, complex):


In [30]:
model_R05.index = pd.to_datetime(model_R05.index)

In [31]:
''' 
arima_R05 = smt.tsa.arima_model.ARIMA(endog = model_R05.DDD_total, order = (2,0,2), dates = model_R05.Období).fit()
fig, ax = plt.subplots()
ax = model_R05.DDD_total.loc['2011-01-01':'2018-03-01'].plot(ax = ax)
fig = arima_R05.plot_predict('2011-01-01', '2018-03-01', ax=ax, dynamic=False, plot_insample=False)
plt.show()
'''

" \narima_R05 = smt.tsa.arima_model.ARIMA(endog = model_R05.DDD_total, order = (2,0,2), dates = model_R05.Období).fit()\nfig, ax = plt.subplots()\nax = model_R05.DDD_total.loc['2011-01-01':'2018-03-01'].plot(ax = ax)\nfig = arima_R05.plot_predict('2011-01-01', '2018-03-01', ax=ax, dynamic=False, plot_insample=False)\nplt.show()\n"

In [32]:
''' 
plot_arima = arima_R05.forecast(steps=10, exog=None, alpha=0.10)[1]
plot_arima = pd.to_numeric(plot_arima)
plot_arima.plot()
plt.show()
'''

' \nplot_arima = arima_R05.forecast(steps=10, exog=None, alpha=0.10)[1]\nplot_arima = pd.to_numeric(plot_arima)\nplot_arima.plot()\nplt.show()\n'

In [33]:
#model_R05.DDD_total

In [34]:
M01.columns

Index(['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [35]:
df = soubor1.drop_duplicates(subset='Období')
df.head(3)

,Období,ATC7,DDD_total
0,2011-01-01,M01AE01,333.0
1124,2011-02-01,M01AE01,1867.0
2199,2011-03-01,M01AE01,500.0


In [36]:
df['Období'] = pd.to_datetime(df['Období'])

C:\Users\Katerina\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
df['trend'] = df['Období'].dt.year.astype(np.int) - 2004
df.head()

C:\Users\Katerina\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Období,ATC7,DDD_total,trend
0,2011-01-01,M01AE01,333.0,7
1124,2011-02-01,M01AE01,1867.0,7
2199,2011-03-01,M01AE01,500.0,7
3300,2011-04-01,M01AE01,1339332.0,7
4333,2011-05-01,M01AE01,2206398.0,7


In [38]:
df = df.reset_index(drop=True)

In [39]:
Mdummies = pd.get_dummies(df['Období'].dt.month.astype(np.int), prefix='month')
Mdummies.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0


In [40]:
model_predict = pd.concat([df, Mdummies], axis=1, join_axes=[df.index])
model_predict.head(3)

,Období,ATC7,DDD_total,trend,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2011-01-01,M01AE01,333.0,7,1,0,0,0,0,0,0,0,0,0,0,0
1,2011-02-01,M01AE01,1867.0,7,0,1,0,0,0,0,0,0,0,0,0,0
2,2011-03-01,M01AE01,500.0,7,0,0,1,0,0,0,0,0,0,0,0,0


In [41]:
model_predict = model_predict.iloc[:, 3:]

In [42]:
model_predict.head(15)

,trend,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,7,1,0,0,0,0,0,0,0,0,0,0,0
1,7,0,1,0,0,0,0,0,0,0,0,0,0
2,7,0,0,1,0,0,0,0,0,0,0,0,0
3,7,0,0,0,1,0,0,0,0,0,0,0,0
4,7,0,0,0,0,1,0,0,0,0,0,0,0
5,7,0,0,0,0,0,1,0,0,0,0,0,0
6,7,0,0,0,0,0,0,1,0,0,0,0,0
7,7,0,0,0,0,0,0,0,1,0,0,0,0
8,7,0,0,0,0,0,0,0,0,1,0,0,0
9,7,0,0,0,0,0,0,0,0,0,1,0,0


In [43]:
pred2018 = model_predict.iloc[3:15, :]
pred2018

,trend,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
3,7,0,0,0,1,0,0,0,0,0,0,0,0
4,7,0,0,0,0,1,0,0,0,0,0,0,0
5,7,0,0,0,0,0,1,0,0,0,0,0,0
6,7,0,0,0,0,0,0,1,0,0,0,0,0
7,7,0,0,0,0,0,0,0,1,0,0,0,0
8,7,0,0,0,0,0,0,0,0,1,0,0,0
9,7,0,0,0,0,0,0,0,0,0,1,0,0
10,7,0,0,0,0,0,0,0,0,0,0,1,0
11,7,0,0,0,0,0,0,0,0,0,0,0,1
12,8,1,0,0,0,0,0,0,0,0,0,0,0


In [44]:
DDDpred = resultsR05.predict(pred2018)
print(DDDpred)

3     3.996574e+06
4     3.281126e+06
5     2.809610e+06
6     2.378276e+06
7     3.458192e+06
8     7.973276e+06
9     6.451424e+06
10    5.757178e+06
11    5.868852e+06
12    8.713636e+06
13    8.978786e+06
14    6.405372e+06
dtype: float64


In [45]:
DDDpred.shape

(12,)

In [46]:
pokus = DDDpred

In [47]:
pokus1 = pd.DataFrame({'month':pokus.index+1, 'DDD_total': pokus.values})

In [48]:
pokus1.loc[:,'year'] = 2018
pokus1

,DDD_total,month,year
0,3.996574e+06,4,2018
1,3.281126e+06,5,2018
2,2.809610e+06,6,2018
3,2.378276e+06,7,2018
4,3.458192e+06,8,2018
5,7.973276e+06,9,2018
6,6.451424e+06,10,2018
7,5.757178e+06,11,2018
8,5.868852e+06,12,2018
9,8.713636e+06,13,2018


In [49]:
pokus1.loc[9:, 'month'] = pokus1['month'] -12
pokus1.loc[9:, 'year'] = 2019

In [50]:
pokus1.loc[:,'day'] = 1

In [51]:
pokus1

,DDD_total,month,year,day
0,3.996574e+06,4,2018,1
1,3.281126e+06,5,2018,1
2,2.809610e+06,6,2018,1
3,2.378276e+06,7,2018,1
4,3.458192e+06,8,2018,1
5,7.973276e+06,9,2018,1
6,6.451424e+06,10,2018,1
7,5.757178e+06,11,2018,1
8,5.868852e+06,12,2018,1
9,8.713636e+06,1,2019,1


In [52]:
pokus1.loc[:,'Období'] =  pd.to_datetime((pokus1.year*10000+pokus1.month*100+pokus1.day).apply(str),format='%Y%m%d')

In [53]:
pokus1

,DDD_total,month,year,day,Období
0,3.996574e+06,4,2018,1,2018-04-01
1,3.281126e+06,5,2018,1,2018-05-01
2,2.809610e+06,6,2018,1,2018-06-01
3,2.378276e+06,7,2018,1,2018-07-01
4,3.458192e+06,8,2018,1,2018-08-01
5,7.973276e+06,9,2018,1,2018-09-01
6,6.451424e+06,10,2018,1,2018-10-01
7,5.757178e+06,11,2018,1,2018-11-01
8,5.868852e+06,12,2018,1,2018-12-01
9,8.713636e+06,1,2019,1,2019-01-01


In [54]:
pokus1 = pokus1.loc[:, ['Období', 'DDD_total']]

In [55]:
R05_DDD = R05_DDD.loc[:, ['Období', 'DDD_total']]

In [56]:
R05_DDD.head()

,Období,DDD_total
0,2011-01-01,8326992.0
1,2011-02-01,8690387.0
2,2011-03-01,6590397.0
3,2011-04-01,3506790.0
4,2011-05-01,3121741.0


In [57]:
pokus1.head()

,Období,DDD_total
0,2018-04-01,3.996574e+06
1,2018-05-01,3.281126e+06
2,2018-06-01,2.809610e+06
3,2018-07-01,2.378276e+06
4,2018-08-01,3.458192e+06


### Grafy:

In [58]:
#prediction = pd.concat([R05_DDD, pokus1], ignore_index=True)

In [59]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [60]:
'''
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x= prediction['Období'], y=prediction['DDD_total'])],
    "layout": go.Layout(title="Spotřeba léků")
})
'''

'\nplotly.offline.init_notebook_mode(connected=True)\n\nplotly.offline.iplot({\n    "data": [go.Scatter(x= prediction[\'Období\'], y=prediction[\'DDD_total\'])],\n    "layout": go.Layout(title="Spotřeba léků")\n})\n'

In [61]:
'''  Example:

trace0 = go.Scatter(
    x= R05_DDD['Období'], y=R05_DDD['DDD_total']
)
trace1 = go.Scatter(
    x= pokus1['Období'], y=pokus1['DDD_total']
)
data = [trace0, trace1]

py.iplot(data, filename = 'basic-line')
'''

"  Example:\n\ntrace0 = go.Scatter(\n    x= R05_DDD['Období'], y=R05_DDD['DDD_total']\n)\ntrace1 = go.Scatter(\n    x= pokus1['Období'], y=pokus1['DDD_total']\n)\ndata = [trace0, trace1]\n\npy.iplot(data, filename = 'basic-line')\n"

In [62]:
plotly.offline.init_notebook_mode(connected=True)

trace0 = go.Scatter(
    x= R05_DDD['Období'], y=R05_DDD['DDD_total']
)
trace1 = go.Scatter(
    x= pokus1['Období'], y=pokus1['DDD_total']
)

data = [trace0, trace1]

plotly.offline.iplot({
    "data": data,
    "layout": go.Layout(title="Spotřeba léků")
})